In [ ]:
# Databricks notebook parameters - these will be passed from ADF
dbutils.widgets.text("user_id", "")
dbutils.widgets.text("parent_job_id", "")
dbutils.widgets.text("entity_type", "user")  # Default to user for this notebook

# Get parameters
user_id = dbutils.widgets.get("user_id")
parent_job_id = dbutils.widgets.get("parent_job_id")
entity_type = dbutils.widgets.get("entity_type")

print(f"Processing {entity_type} data for user {user_id}, batch {parent_job_id}")

# Validate required parameters
if not user_id or not parent_job_id:
    raise ValueError("user_id and parent_job_id are required parameters")

import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def get_optimal_partition_col(df, candidates):
    """
    Selects the best partition column based on cardinality.
    Dynamically calculates target partitions based on data size for optimal performance.
    """
    num_rows = df.count()
    target_partitions = max(4, min(50, num_rows // 20000))  # Adjust divisor as needed based on row size
    
    best_col = None
    best_diff = float('inf')    
    for col in candidates:
        if col not in df.columns:
            continue
            
        cardinality = df.select(col).distinct().count()        
        if 1 < cardinality <= 100:
            diff = abs(cardinality - target_partitions)
            if diff < best_diff:
                best_diff = diff
                best_col = col
                
    return best_col




# COMMAND ----------
from pyspark.sql.functions import col, current_timestamp, lit, lower, trim, regexp_replace
print(f'Starting {entity_type} table processing...')

# Dynamic configuration based on parameters
ADLS_ACCOUNT_NAME = "shanleestorage"  # Your storage account name

# Bronze layer paths (source)
RAW_CONTAINER = "shanlee-raw-data"
RAW_DATA_PATH = f"{user_id}/{parent_job_id}"  # Dynamic path based on user/batch
RAW_FULL_PATH = f"abfss://{RAW_CONTAINER}@{ADLS_ACCOUNT_NAME}.dfs.core.windows.net/{RAW_DATA_PATH}"

# Silver layer paths (destination)
SILVER_CONTAINER = "shanlee-cleaned-data"
SILVER_PATH = f"silver/cleaned/{user_id}/{parent_job_id}/{entity_type}"
SILVER_FULL_PATH = f"abfss://{SILVER_CONTAINER}@{ADLS_ACCOUNT_NAME}.dfs.core.windows.net/{SILVER_PATH}"

print(f"Reading from: {RAW_FULL_PATH}")
print(f"Writing to: {SILVER_FULL_PATH}")

# Authentication (same as before)
SECRET_SCOPE = "AdlsAccessKey"    
SECRET_KEY = "AdlsAccessKey"

try:
    access_key_value = dbutils.secrets.get(scope=SECRET_SCOPE, key=SECRET_KEY)
    
    spark.conf.set(
        f"fs.azure.account.key.{ADLS_ACCOUNT_NAME}.dfs.core.windows.net",
        access_key_value
    )
    
    print("Authentication successful: Spark configured to access ADLS Gen2.")

except Exception as e:
    print(f"FATAL ERROR: Could not retrieve secret. Check scope/key names. Error: {e}")
    dbutils.notebook.exit("Authentication Failed")

# Read raw data from Bronze layer
df_raw = spark.read.format("json") \
             .option("multiline", "true") \
             .load(RAW_FULL_PATH)

# Extract the specific entity table from the raw data
entity_table = df_raw.select(f"{entity_type}.*")

print(f"Loaded {entity_table.count()} raw {entity_type} records")

# Apply entity-specific transformations
if entity_type == "user":
    # User-specific cleaning logic
    entity_table = entity_table.withColumn("real_name", F.trim(F.lower(F.col("real_name"))))
    entity_table = entity_table.withColumn("company", F.trim(F.lower(F.col("company"))))
    entity_table = entity_table.withColumn("job", F.trim(F.lower(F.col("job"))))
    entity_table = entity_table.dropDuplicates(["id"])

    # Filter out rows containing 'invalid' in any column
    for column in entity_table.columns:
        entity_table = entity_table.filter(~F.lower(F.col(column).cast("string")).contains("invalid"))

    # Filter sex: ensure it's one of 'male', 'female', or 'other' (case-insensitive, trimmed)
    entity_table = entity_table.filter(F.lower(F.trim(F.col("sex"))).isin(["male", "female", "other"]))

    # Filter age: ensure it's numeric and within valid range (10-100)
    entity_table = entity_table.withColumn("age", F.col("age").cast("int")) \
        .filter(F.col("age").isNotNull() & (F.col("age") >= 10) & (F.col("age") <= 100))

    # Filter birth_of_date: ensure it's a valid date and less than today
    entity_table = entity_table.withColumn("birth_of_date", F.to_date(F.col("birth_of_date"))) \
        .filter(F.col("birth_of_date").isNotNull() & (F.col("birth_of_date") < F.current_date()))

    # Remove duplicates based on real_name
    entity_table = entity_table.dropDuplicates(["real_name"])

elif entity_type == "address":
    # Address-specific cleaning logic
    entity_table = entity_table.withColumn("street", F.trim(F.lower(F.col("street"))))
    entity_table = entity_table.withColumn("city", F.trim(F.lower(F.col("city"))))
    entity_table = entity_table.withColumn("country", F.trim(F.lower(F.col("country"))))
    entity_table = entity_table.dropDuplicates(["id"])
    
    # Add your address validation logic here
    
elif entity_type == "product":
    # Product-specific cleaning logic
    entity_table = entity_table.withColumn("name", F.trim(F.lower(F.col("name"))))
    entity_table = entity_table.withColumn("category", F.trim(F.lower(F.col("category"))))
    entity_table = entity_table.dropDuplicates(["id"])
    
    # Add your product validation logic here

# Add more entity types as needed...

else:
    print(f"Warning: No specific cleaning logic defined for entity type '{entity_type}'. Applying basic deduplication only.")
    entity_table = entity_table.dropDuplicates()

# COMMAND ----------
# --- 5. LOAD TO SILVER LAYER: Write Cleaned Data as Delta Lake ---

# Use Delta format for reliability, transactions, and schema enforcement.

writer = entity_table.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true")

writer.save(SILVER_FULL_PATH)

print(f"✅ Cleaned {entity_type} data successfully saved to Silver layer at: {SILVER_FULL_PATH}")
print(f"Processed {entity_table.count()} cleaned records")
print("Next Steps: Gold layer processing will handle aggregations and joins.")